# Bioconductor and Differential Expression

## 1.	Install bioconductor on your computer

1. Troubleshooting tips from bioconductor http://bioconductor.org/install/
2. Download the source of the package from bioconductor and then use "install.package" in R

In [ ]:
# The installation instructions from http://bioconductor.org/install/ (version 3.16 of Bioconductor requires R 4.2)
# Since the UW JupyterHub has R version 4.1, we are installing version 3.14 of Bioconductor instead.
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.14")

Bioconductor version '3.14' is out-of-date; the current release version '3.16'
  is available with R version '4.2'; see https://bioconductor.org/install

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.19), R 4.1.1 (2021-08-10)

Old packages: 'arkhe', 'brew', 'broom', 'caret', 'class', 'classInt', 'cli',
  'codetools', 'colorspace', 'conflicted', 'covr', 'credentials', 'curl',
  'data.table', 'dbplyr', 'devtools', 'dplyr', 'DT', 'e1071', 'evaluate',
  'expm', 'fansi', 'FNN', 'fontawesome', 'forcats', 'forecast', 'fracdiff',
  'fs', 'future', 'gargle', 'gert', 'ggplot2', 'ggrepel', 'gh', 'git2r',
  'gitcreds', 'gower', 'gstat', 'gtools', 'highr', 'htmlwidgets', 'httpuv',
  'IRdisplay', 'IRkernel', 'isoband', 'janitor', 'keras', 'knitr', 'lava',
  'lhs', 'lmtest', 'lubridate', 'markdown', 'MASS', 'Matrix', 'mgcv',
  'modeldata', '

## 2.	Download the CCLE data from GEO

Go to GEO’s web site at http://www.ncbi.nlm.nih.gov/geo/.
1.	Search for CCLE.
2.	Click on the first link (GSE36133)
3.	http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE36133

In [ ]:
BiocManager::install("GEOquery")
library(GEOquery)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.19), R 4.1.1 (2021-08-10)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'GEOquery'”
Old packages: 'arkhe', 'brew', 'broom', 'caret', 'class', 'classInt', 'cli',
  'codetools', 'colorspace', 'conflicted', 'covr', 'credentials', 'curl',
  'data.table', 'dbplyr', 'devtools', 'dplyr', 'DT', 'e1071', 'evaluate',
  'expm', 'fansi', 'FNN', 'fontawesome', 'forcats', 'forecast', 'fracdiff',
  'fs', 'future', 'gargle', 'gert', 'ggplot2', 'ggrepel', 'gh', 'git2r',
  'gitcreds', 'gower', 'gstat', 'gtools', 'highr', 'htmlwidgets', 'httpuv',
  'IRdisplay', 'IRkernel', 'isoband', 'janitor', 'keras', 'knitr', 'lava',
  'lhs', 'lmtest', 'lubridate', 'markdown', 'MASS', 'Matrix', 'mgcv',
  'modeldata', 'nlme', 'nloptr

In [7]:
# this is going to take a few minutes.  
# This command assumes that you can install the package GEOquery.
# if this package doesn't work for you, you can load my image in your workspace.
geoD<- getGEO('GSE36133')

Found 1 file(s)

GSE36133_series_matrix.txt.gz

Using locally cached version: /tmp/Rtmp5B8sKd/GSE36133_series_matrix.txt.gz

Using locally cached version of GPL15308 found here:
/tmp/Rtmp5B8sKd/GPL15308.soft.gz 



In [8]:
# get the expression data from geoD
mat <- exprs (geoD[[1]])
dim (mat)

[1] 18926   917

#### The gene expression data "mat" consists of 18926 genes across 917 experiments
#### Each row is a gene
#### Each column is an experiment (cell line)

In [9]:
mat[1:6, 1:5]

,GSM886835,GSM886836,GSM886837,GSM886838,GSM886839
100009676_at,6.1161,6.2052,6.1249,6.6154,5.4236
10000_at,8.1556,6.6152,4.5676,4.3519,6.6723
10001_at,9.7864,9.9699,8.8720,9.1376,10.0290
10002_at,3.7977,4.0304,3.8455,3.7085,3.6431
10003_at,3.5458,3.8504,4.0458,3.9508,4.1589
100048912_at,4.0034,3.7959,4.1465,3.9271,3.9157


In [10]:
#get phenotype matrix from geoD
p.mat <- pData (geoD[[1]])
dim (p.mat)

[1] 917  42

#### The phenotype matrix consists of 917 experiments, and 42 attributes
#### Rows = experiments (cell lines)

In [11]:
p.mat[1:6, c(1:2, 10)]

,title,geo_accession,characteristics_ch1
,<chr>,<chr>,<chr>
GSM886835,1321N1,GSM886835,primary site: central_nervous_system
GSM886836,143B,GSM886836,primary site: bone
GSM886837,22Rv1,GSM886837,primary site: prostate
GSM886838,23132/87,GSM886838,primary site: stomach
GSM886839,42-MG-BA,GSM886839,primary site: central_nervous_system
GSM886840,5637,GSM886840,primary site: urinary_tract


In [12]:
# goal: to extract the character string after ": "
unlist(strsplit (as.character (p.mat[1,10]), ": "))[2]

[1] "central_nervous_system"

In [13]:
# extract the character string after ": " for each entry in column 10 of p.mat
source.vec <- sapply (as.character(p.mat[, 10]), function(x) {unlist(strsplit(x, ": "))[2]})

In [14]:
source.vec[1:10]

primary site: central_nervous_system 
                        "central_nervous_system" 
                              primary site: bone 
                                          "bone" 
                          primary site: prostate 
                                      "prostate" 
                           primary site: stomach 
                                       "stomach" 
            primary site: central_nervous_system 
                        "central_nervous_system" 
                     primary site: urinary_tract 
                                 "urinary_tract" 
                     primary site: urinary_tract 
                                 "urinary_tract" 
                     primary site: urinary_tract 
                                 "urinary_tract" 
primary site: haematopoietic_and_lymphoid_tissue 
            "haematopoietic_and_lymphoid_tissue" 
                            primary site: kidney 
                                        "kidney"

In [15]:
as.character(source.vec[1:10])

[1] "central_nervous_system"             "bone"                              
 [3] "prostate"                           "stomach"                           
 [5] "central_nervous_system"             "urinary_tract"                     
 [7] "urinary_tract"                      "urinary_tract"                     
 [9] "haematopoietic_and_lymphoid_tissue" "kidney"

In [16]:
# get all the unique cell lines
unique(as.character(source.vec))

[1] "central_nervous_system"             "bone"                              
 [3] "prostate"                           "stomach"                           
 [5] "urinary_tract"                      "haematopoietic_and_lymphoid_tissue"
 [7] "kidney"                             "thyroid"                           
 [9] "skin"                               "soft_tissue"                       
[11] "salivary_gland"                     "ovary"                             
[13] "lung"                               "pleura"                            
[15] "liver"                              "endometrium"                       
[17] "pancreas"                           "breast"                            
[19] "upper_aerodigestive_tract"          "large_intestine"                   
[21] "autonomic_ganglia"                  "oesophagus"                        
[23] "biliary_tract"                      "small_intestine"

In [17]:
# sanity check: how many experiments do we have?
length (source.vec)

[1] 917

In [18]:
# how many unique cell lines do we have?
unique.source.vec <- unique (as.character(source.vec))
length(unique (as.character(source.vec)))

[1] 24

In [15]:
# how many cell lines are of the type "lung"?
sum (as.character(source.vec)=="lung")

[1] 166

## Your task: how to compute the count for each unique cell line

In [16]:
unique.source.vec <- unique(as.character(source.vec))
count.per.source <- sapply (unique.source.vec, function (x) {sum(as.character(source.vec) == x)})
count.per.source

central_nervous_system                               bone 
                                47                                 25 
                          prostate                            stomach 
                                 7                                 34 
                     urinary_tract haematopoietic_and_lymphoid_tissue 
                                22                                172 
                            kidney                            thyroid 
                                20                                 11 
                              skin                        soft_tissue 
                                58                                 16 
                    salivary_gland                              ovary 
                                 2                                 44 
                              lung                             pleura 
                               166                                  9 
                             liver                        endometrium 
                                26                                 25 
                          pancreas                             breast 
                                43                                 56 
         upper_aerodigestive_tract                    large_intestine 
                                29                                 55 
                 autonomic_ganglia                         oesophagus 
                                17                                 25 
                     biliary_tract                    small_intestine 
                                 7                                  1

In [17]:
# get the indices of "bone" cell lines
bone.ind <- which (as.character(source.vec) == "bone")
# get the indices of "oesophagus" cell lines
oesophagus.ind <- which (as.character(source.vec) == "oesophagus")

In [18]:
# create a new matrix consisting of all the bone cell lines, followed by the oesophagus cell lines
mat.sub <- mat[, c(bone.ind, oesophagus.ind)]
dim (mat.sub)

[1] 18926    50

### first 25 columns in mat.sub correspond to "bone" samples, second 25 columns correspond to "oesophagus"

In [19]:
write.table (mat.sub, file="CCLE_expr_bone_oesophagus.txt", sep="\t", quote=F, row.names=T, col.names=T)

In [20]:
bone.ind

[1]   2  25  67  78 181 254 276 278 284 285 286 290 291 292 480 484 646 693 740
[20] 743 762 832 848 855 888

In [21]:
oesophagus.ind

[1] 111 162 413 414 415 416 417 418 419 420 421 422 641 642 857 858 859 860 861
[20] 863 864 866 867 868 879

In [22]:
c(bone.ind, oesophagus.ind)

[1]   2  25  67  78 181 254 276 278 284 285 286 290 291 292 480 484 646 693 740
[20] 743 762 832 848 855 888 111 162 413 414 415 416 417 418 419 420 421 422 641
[39] 642 857 858 859 860 861 863 864 866 867 868 879

## Differential expression using the t-test

For each gene A, is the mean of this gene under the bone samples different from the mean of this gene under the oesophagus samples?

In [23]:
# Apply the t-test to the first row (gene), comparing the distributions of the bone cell lines versus the oesophagus cell lines
t.test (mat.sub[1, 1:25], mat.sub[1, 26:50])


	Welch Two Sample t-test

data:  mat.sub[1, 1:25] and mat.sub[1, 26:50]
t = -0.67926, df = 39.2, p-value = 0.501
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.2862871  0.1423271
sample estimates:
mean of x mean of y 
 5.672396  5.744376 


### performed t-test for 1 gene

In [24]:
# extract the p-value from the previous t-test
t.test (mat.sub[1, 1:25], mat.sub[1, 26:50])$p.value

[1] 0.5009665

### for the first gene, we want to ask of the mean of the first gene is different under bones (first 25 samples 1:25) than under oesophagus (the second 25 samples 26:50)

In [27]:
p.vec <- apply (mat.sub, 1, function (x) {t.test(x[1:25], x[26:50])$p.value})

In [28]:
sum (p.vec < 0.05)

[1] 7495

In [29]:
sum (p.vec < 0.01)

[1] 5050

In [30]:
sum (p.vec < 0.0000001)

[1] 507

### The more stringent p-value threshold will give you fewer differentially expressed genes.

### For each gene, reject the null hypotheis (the means are the same) --> the gene is differentially expressed.

## Question: should we expect so many differentially expressed genes?

### Bonferroni correction: p.value.threshold/(# independent t-tests)

In [31]:
help (p.adjust)

p.adjust {stats},R Documentation
p,numeric vector of p-values (possibly with NAs). Any other R object is coerced by as.numeric.
method,"correction method, a character string. Can be abbreviated."
n,"number of comparisons, must be at least length(p); only set this (to non-default) when you know what you are doing!"


In [32]:
adj.p.vec1 <- p.adjust (p.vec, method="bonferroni")

In [33]:
sum (adj.p.vec1 < 0.05)

[1] 904

In [34]:
new.alpha <- 0.05 / length(p.vec)
new.alpha

[1] 2.641868e-06

In [35]:
sum (p.vec < new.alpha)

[1] 904

## create a smaller dataset for future analysis